Cifar10

Download data

In [1]:
from torchvision.datasets import CIFAR10

cifar10_trainset = CIFAR10("./data/cifar10/", train=True, download=True)
cifar10_testset = CIFAR10("./data/cifar10/", train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


Split data into train, validation and test sets

In [2]:
import numpy as np
from src import preprocess

validation_percent = 0.2
one_hot_encoder = preprocess.OneHotEncoder()


train_size = int((1 - validation_percent) * len(cifar10_trainset))
validation_size = len(cifar10_trainset) - train_size

x_train = cifar10_trainset.data[:train_size].reshape(train_size, -1)
targets_train = np.array(cifar10_trainset.targets[:train_size]).reshape(train_size,1)
y_train = one_hot_encoder.transform(targets_train)

x_valid = cifar10_trainset.data[train_size:].reshape(validation_size, -1)
targets_valid = np.array(cifar10_trainset.targets[train_size:]).reshape(validation_size,1)
y_valid = one_hot_encoder.transform(targets_valid)

x_test = cifar10_testset.data.reshape(len(cifar10_testset), -1)
targets_test = np.array(cifar10_testset.targets).reshape(len(cifar10_testset),1)
y_test = one_hot_encoder.transform(targets_test)

print(type(x_train), type(y_train))
print(x_train.shape, y_train.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(40000, 3072) (40000, 10)


Setup neural network, loss function, optimizer and logger

In [3]:
from src import (
    FFNN,
    layer,
    activation_functions,
    initializers,
    loss_functions,
    optimizers,
)


activation_function = activation_functions.ReLU()
identity_activation = activation_functions.Identity()

weight_init = initializers.HeInitializer()
bias_init = initializers.ConstantInitializer(0.0)

loss_fn = loss_functions.CrossEntropyLoss()
optimizer = optimizers.SGD(learning_rate=0.01)

layers = [layer.Layer(input_size=32 * 32 * 3, output_size=128, activation_function=activation_function, weight_initializer=weight_init, bias_initializer=bias_init), 
          layer.Layer(input_size=128, output_size=64, activation_function=activation_function, weight_initializer=weight_init, bias_initializer=bias_init), 
          layer.Layer(input_size=64, output_size=10, activation_function=identity_activation, weight_initializer=weight_init, bias_initializer=bias_init)]

model = FFNN.FFNN(layers=layers, loss_function=loss_fn, optimizer=optimizer)

Training the model

In [4]:
from src import training, wandblogger

epochs = 10
batch_size = 64



trainer = training.Trainer(model=model, loss_function=loss_fn)


history = trainer.train(
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True)

Epoch 1/10 - Loss: 155.3120
Epoch 2/10 - Loss: 155.3120
Epoch 3/10 - Loss: 155.3120
Epoch 4/10 - Loss: 155.3120
Epoch 5/10 - Loss: 155.3120


KeyboardInterrupt: 